In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/big-data/big_data_3468_5202.csv
/kaggle/input/big-data/big_data_5202_6936.csv
/kaggle/input/big-data/big_data_1734_3468.csv
/kaggle/input/big-data/big_data_0_1734.csv


In [72]:
df = pd.read_csv('/kaggle/input/big-data/big_data_0_1734.csv')
df.columns
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [73]:
def get_str_desc(row):
    return str(row['concatenated_desc'])

In [74]:
print(df.columns)
print(df.shape[0])

df['concatenated_desc'] = df.apply(lambda x : get_str_desc(x), axis=1)
# print(df.head()['concatenated_desc'])

description_length = [len(i) for i in list(df['concatenated_desc'])]
print(max(description_length))
print(sum(description_length)/len(description_length))
# description_list = list(df['final_description'])

Index(['zpid', 'bedrooms', 'bathrooms', 'price', 'zipcode', 'hdpUrl', 'cityId',
       'homeType', 'livingArea', 'livingAreaValue', 'rentZestimate',
       'description', 'latitude', 'longitude', 'homeInsights', 'photoCount',
       'originalPhotos', 'address.streetAddress', 'address.city',
       'address.state', 'address.zipcode', 'image_captions',
       'leisure_within_5km', 'shops_within_5km', 'schools_within_5km',
       'transit_within_2km', 'concatenated_desc'],
      dtype='object')
1734
14603
3299.253748558247


In [75]:
# maxi = 0
# sum = 0;
# for i in range(len(description_list)):
# #     print(i)
# #     break;
#     if(len(str(description_list[i]))>maxi):
#         idx = i;
#     sum+=len(str(description_list[i]));
#     maxi = max(len(str(description_list[i])),maxi)
# print(maxi)
# print(sum/len(description_list))
# # print(description_list[idx])


def split_string(string, block_size):
    return [string[i:i+block_size] for i in range(0, len(string), block_size)]

def generate_summary(row):
    
    text = row['concatenated_desc']
#     print(text)
    list_of_texts = split_string(text, 1024)
#     print(list_of_texts)
    summaries = [summarizer(text,max_length=80,min_length=50,do_sample=False) for text in list_of_texts]
    summaries = [summary[0]['summary_text'] for summary in summaries]
    
#     print(summaries)
#     summary = summarizer(text,max_length=80,min_length=50,do_sample=False)
    return ''.join(summaries)

#     final_description_list.append(summary)

In [76]:
# print(df.head(15))
# df = df.iloc[13:14]
df['summary'] = df.apply(lambda x : generate_summary(x), axis=1)
# print(df['summary'][13])

Beautiful & grand Victorian home with distinctive architectural features. Original crown molding, multiple fireplaces, original heart pine flooring, and grandeur associated only with certain homes built before 1900. Voluminous main level spaces perfect for entertaining. Dramatic central staircase leading to upper levels with plenty of bedroom space.A row of brick homes with a sunset in the background and a fenced in. Transit oriented location with Metro buses on 18th Street and Metros nearby (0.7miles to Dupont and U St/Cardozo). Estate Sale; property conveys As-Is.A kitchen with a large island and a television mounted above it's cabinet doors. A bedroom with a bed, dresser, and chair in it with a window in. A row of brick townhouses with a fence in front of them and a tree. A living room with a fireplace and a chair and a table with a clock on.A bedroom with a bed, chair, fireplace and a television in it's. A bathroom with a tub, toilet, and sink in it and a window in it. A kitchen wi

In [79]:
# print(len(df['summary'][13]))

1375


In [77]:
# df['Summarized_description'] = final_description_list

In [78]:
df.to_csv("FinalDataset_new.csv")